In [1]:
import random

# from sentence_transformers import SentenceTransformer
import pandas as pd
# import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np
import json
import joblib
import requests

from sacrebleu.metrics import BLEU
from rouge import Rouge
import statistics

make sure server is on and the pkl files of kmeans clustes is available.

In [2]:
weights = [5,5,1,1,1,1,1,1,1]
cols = ['bedrooms', 'bathrooms', 'price', 'livingArea', 'leisure_within_5km', 'shops_within_5km', 'schools_within_5km', 'transit_within_2km', 'homeType']

Running on Sample Input

In [3]:
state = 'NY'
kmeans = joblib.load(f"{state}_kmeans.pkl")
scaler = joblib.load(f"{state}_scaler.pkl")

In [4]:
cluster_centers = kmeans.cluster_centers_

# Since we applied weights, we need to reverse this before denormalization
cluster_centers_unweighted = cluster_centers /weights

cluster_centers_to_denorm = cluster_centers_unweighted[:,:-1]
# print(cluster_centers_to_denorm)
# Denormalize the cluster centers
cluster_centers_denormalized = scaler.inverse_transform(cluster_centers_to_denorm)

cluster_centers_final= np.concatenate((cluster_centers_denormalized,cluster_centers_unweighted[:,-1].reshape(-1, 1)),axis=1)
cluster_centers_final

array([[ 3.03846154e+00,  2.40384615e+00,  2.42096154e+06,
         7.99519135e+02,  2.25769231e+01,  5.78653846e+02,
         1.72846154e+02,  2.78788462e+02,  1.00000000e+00],
       [ 1.00000000e+00,  1.13017751e+00,  1.03582721e+06,
         4.28289823e+02,  2.30118343e+01,  6.30177515e+02,
         1.72775148e+02,  2.91431953e+02,  9.64497041e-01],
       [ 4.43478261e+00,  5.04347826e+00,  8.17304348e+06,
         3.67691304e+03,  2.11304348e+01,  6.17782609e+02,
         1.72391304e+02,  2.82913043e+02,  1.82608696e+00],
       [ 2.00000000e+00,  2.00000000e+00,  1.99031818e+06,
         8.01519481e+02,  2.33116883e+01,  5.98233766e+02,
         1.71506494e+02,  2.84051948e+02,  9.61038961e-01],
       [ 6.10000000e+00,  7.50000000e+00,  1.49290000e+07,
         4.93440000e+03,  2.30000000e+01,  5.82000000e+02,
         1.72100000e+02,  2.82900000e+02,  1.90000000e+00],
       [ 0.00000000e+00,  1.02173913e+00,  6.05358587e+05,
         2.59347610e+02,  2.28913043e+01,  6.482608

In [5]:
df_cluster_centers=pd.DataFrame(cluster_centers_final,columns=list(cols))
np.floor(df_cluster_centers)

,bedrooms,bathrooms,price,livingArea,leisure_within_5km,shops_within_5km,schools_within_5km,transit_within_2km,homeType
0,3.0,2.0,2420961.0,799.0,22.0,578.0,172.0,278.0,1.0
1,1.0,1.0,1035827.0,428.0,23.0,630.0,172.0,291.0,0.0
2,4.0,5.0,8173043.0,3676.0,21.0,617.0,172.0,282.0,1.0
3,2.0,2.0,1990318.0,801.0,23.0,598.0,171.0,284.0,0.0
4,6.0,7.0,14929000.0,4934.0,23.0,582.0,172.0,282.0,1.0
5,0.0,1.0,605358.0,259.0,22.0,648.0,169.0,294.0,0.0
6,1.0,1.0,1105764.0,269.0,-1.0,0.0,0.0,0.0,1.0
7,1.0,3.0,3290223.0,1361.0,23.0,632.0,173.0,317.0,1.0
8,9.0,5.0,1999500.0,3860.0,44.0,245.0,194.0,254.0,3.0
9,2.0,12.0,3980000.0,1283.0,21.0,638.0,165.0,348.0,1.0


In [6]:
predict_X=[2,1,1700000,2000,20,400,250,200,2]  # give input here
description = "A large backyard, swimming pool, spacious living room, furniture in bedroom, tiled bathroom"
df_predict = pd.DataFrame([predict_X],columns=cols)
scaler_predict=StandardScaler()

df_to_transform_p = df_predict.drop(columns=['homeType'])
df_excluded_p = df_predict[['homeType']]
# Apply the transformation
transformed_array_p = scaler.transform(df_to_transform_p)
print(transformed_array_p)
df_transformed_p = pd.DataFrame(transformed_array_p, columns=df_to_transform_p.columns, index=df_predict.index)
# Recombine the DataFrame
df_result_p = pd.concat([df_transformed_p, df_excluded_p], axis=1)
# print(df_result)
X_p=df_result_p.values
X_p=X_p*weights
X_p

[[ 0.09090596 -0.73108393 -0.20746602  0.8198349  -0.35234394 -0.98294832
   2.22322311 -0.88709456]]


array([[ 0.45452982, -3.65541963, -0.20746602,  0.8198349 , -0.35234394,
        -0.98294832,  2.22322311, -0.88709456,  2.        ]])

In [7]:
kmeans.predict(X_p)

array([11], dtype=int32)

Running on Server Response

In [8]:
homeType_Encoding= {
        'APARTMENT': 0,
        'CONDO': 1,
        'MANUFACTURED': 2,
        'MULTI_FAMILY': 3,
        'SINGLE_FAMILY': 4,
        'TOWNHOUSE': 5
    }

In [9]:
state = 'NY'
kmeans = joblib.load(f"{state}_kmeans.pkl")
scaler = joblib.load(f"{state}_scaler.pkl")
weights = [5,5,1,1,1,1,1,1,1]


In [10]:
json_file_path = "./ClusterPoints_Dataset.json"

with open(json_file_path, "r") as file:
    kmeansdata = json.load(file)

json_file_path_1 = "./Labelled_Dataset_with_zpid.json"

with open(json_file_path_1, "r") as file:
    fulldata = json.load(file)

In [11]:
predict_X=[2,1,1700000,2000,20,400,250,200,2]  # give input here
df_predict = pd.DataFrame([predict_X],columns=cols)
scaler_predict=StandardScaler()

df_to_transform_p = df_predict.drop(columns=['homeType'])
df_excluded_p = df_predict[['homeType']]
# Apply the transformation
transformed_array_p = scaler.transform(df_to_transform_p)
print(transformed_array_p)
df_transformed_p = pd.DataFrame(transformed_array_p, columns=df_to_transform_p.columns, index=df_predict.index)
# Recombine the DataFrame
df_result_p = pd.concat([df_transformed_p, df_excluded_p], axis=1)
# print(df_result)
X_p=df_result_p.values
X_p=X_p*weights
X_p

[[ 0.09090596 -0.73108393 -0.20746602  0.8198349  -0.35234394 -0.98294832
   2.22322311 -0.88709456]]


array([[ 0.45452982, -3.65541963, -0.20746602,  0.8198349 , -0.35234394,
        -0.98294832,  2.22322311, -0.88709456,  2.        ]])

In [12]:
predicted_label=kmeans.predict(X_p)
predicted_label

array([11], dtype=int32)

In [13]:
state_full_data=fulldata[state]
state_full_data

[{'zpid': 31491229,
  'bedrooms': 1,
  'bathrooms': 1,
  'price': 650000.0,
  'homeType': 1,
  'livingArea': 776.0,
  'leisure_within_5km': 21,
  'shops_within_5km': 540,
  'schools_within_5km': 128,
  'transit_within_2km': 121,
  'labels': 1},
 {'zpid': 31492996,
  'bedrooms': 2,
  'bathrooms': 2,
  'price': 959000.0,
  'homeType': 1,
  'livingArea': 1000.0,
  'leisure_within_5km': 19,
  'shops_within_5km': 538,
  'schools_within_5km': 127,
  'transit_within_2km': 110,
  'labels': 3},
 {'zpid': 31493494,
  'bedrooms': 1,
  'bathrooms': 1,
  'price': 1100000.0,
  'homeType': 1,
  'livingArea': 815.0,
  'leisure_within_5km': 20,
  'shops_within_5km': 614,
  'schools_within_5km': 141,
  'transit_within_2km': 194,
  'labels': 1},
 {'zpid': 31494186,
  'bedrooms': 0,
  'bathrooms': 1,
  'price': 750000.0,
  'homeType': 1,
  'livingArea': 465.0,
  'leisure_within_5km': 22,
  'shops_within_5km': 717,
  'schools_within_5km': 160,
  'transit_within_2km': 272,
  'labels': 5},
 {'zpid': 31494718

In [14]:
def filter_by_labels(data, labels):
    # Use list comprehension to get dictionaries with matching labels
    filtered_data = [entry for entry in data if entry['labels'] == labels]
    return filtered_data

In [15]:
state_filtered_data = filter_by_labels(state_full_data, predicted_label[0])
len(state_filtered_data)

27

In [16]:
state_filtered_data

[{'zpid': 31495264,
  'bedrooms': 2,
  'bathrooms': 1,
  'price': 798000.0,
  'homeType': 1,
  'livingArea': 525.0,
  'leisure_within_5km': 28,
  'shops_within_5km': 836,
  'schools_within_5km': 181,
  'transit_within_2km': 268,
  'labels': 11},
 {'zpid': 31507674,
  'bedrooms': 2,
  'bathrooms': 1,
  'price': 1350000.0,
  'homeType': 1,
  'livingArea': 1053.0,
  'leisure_within_5km': 21,
  'shops_within_5km': 762,
  'schools_within_5km': 182,
  'transit_within_2km': 346,
  'labels': 11},
 {'zpid': 31527126,
  'bedrooms': 2,
  'bathrooms': 1,
  'price': 875000.0,
  'homeType': 1,
  'livingArea': 0.0,
  'leisure_within_5km': 21,
  'shops_within_5km': 712,
  'schools_within_5km': 170,
  'transit_within_2km': 403,
  'labels': 11},
 {'zpid': 31545643,
  'bedrooms': 2,
  'bathrooms': 1,
  'price': 1050000.0,
  'homeType': 1,
  'livingArea': 0.0,
  'leisure_within_5km': 23,
  'shops_within_5km': 683,
  'schools_within_5km': 176,
  'transit_within_2km': 294,
  'labels': 11},
 {'zpid': 3154675

Sorting Entries based on distance from cluster

In [17]:
df = pd.DataFrame(state_filtered_data)
df=df.set_index(keys=['zpid'])
df

,bedrooms,bathrooms,price,homeType,livingArea,leisure_within_5km,shops_within_5km,schools_within_5km,transit_within_2km,labels
zpid,,,,,,,,,,
31495264,2,1,798000.0,1,525.000000,28,836,181,268,11
31507674,2,1,1350000.0,1,1053.000000,21,762,182,346,11
31527126,2,1,875000.0,1,0.000000,21,712,170,403,11
31545643,2,1,1050000.0,1,0.000000,23,683,176,294,11
31546758,2,1,649000.0,1,755.000000,29,492,212,246,11
61968154,2,1,1590000.0,1,884.000000,24,659,186,300,11
72532295,2,1,799000.0,1,0.000000,25,543,209,272,11
72534234,3,1,885000.0,1,1212.000000,28,402,205,345,11
72535178,2,1,1049000.0,1,0.000000,24,350,185,216,11


In [18]:
df_data = df.drop(columns=['labels'])
df_home= df_data['homeType']
df_data=df_data.drop(columns=['homeType'])
df_final = pd.concat((df_data, df_home),axis=1)
df_final

,bedrooms,bathrooms,price,livingArea,leisure_within_5km,shops_within_5km,schools_within_5km,transit_within_2km,homeType
zpid,,,,,,,,,
31495264,2,1,798000.0,525.000000,28,836,181,268,1
31507674,2,1,1350000.0,1053.000000,21,762,182,346,1
31527126,2,1,875000.0,0.000000,21,712,170,403,1
31545643,2,1,1050000.0,0.000000,23,683,176,294,1
31546758,2,1,649000.0,755.000000,29,492,212,246,1
61968154,2,1,1590000.0,884.000000,24,659,186,300,1
72532295,2,1,799000.0,0.000000,25,543,209,272,1
72534234,3,1,885000.0,1212.000000,28,402,205,345,1
72535178,2,1,1049000.0,0.000000,24,350,185,216,1


In [19]:
np.unique(df_final['bathrooms'])

array([1])

In [20]:
df_to_transform = df_final.drop(columns=['homeType'])
df_excluded= df_final[['homeType']]
# Apply the transformation
transformed_array = scaler.transform(df_to_transform)
df_transformed = pd.DataFrame(transformed_array, columns=df_to_transform.columns, index=df_final.index)
# Recombine the DataFrame
df_result= pd.concat([df_transformed, df_excluded], axis=1)
# print(df_result)
X=df_result.values
X=X*weights
X[:,1]

array([-3.65541963, -3.65541963, -3.65541963, -3.65541963, -3.65541963,
       -3.65541963, -3.65541963, -3.65541963, -3.65541963, -3.65541963,
       -3.65541963, -3.65541963, -3.65541963, -3.65541963, -3.65541963,
       -3.65541963, -3.65541963, -3.65541963, -3.65541963, -3.65541963,
       -3.65541963, -3.65541963, -3.65541963, -3.65541963, -3.65541963,
       -3.65541963, -3.65541963])

In [21]:
cluster_centers = kmeans.cluster_centers_
cluster_of_interest = cluster_centers[predicted_label[0]]
cluster_of_interest

array([ 0.72605301, -3.65541963, -0.43643778, -0.19942973,  0.64564389,
       -0.32447643,  0.34361588, -0.11963628,  0.96296296])

In [22]:
X = np.array(X)

Y = np.array(cluster_of_interest)

# Calculate Euclidean distances
distances = np.linalg.norm(X - Y, axis=1)

# Get the indices that would sort the distances
sorted_indices = np.argsort(distances)
row_indexes = df_final.index[sorted_indices]
# Sort the 2D matrix based on the distances
sorted_X = X[sorted_indices]


In [23]:
print("\nSorted matrix:")
print(sorted_X[:,1])


Sorted matrix:
[-3.65541963 -3.65541963 -3.65541963 -3.65541963 -3.65541963 -3.65541963
 -3.65541963 -3.65541963 -3.65541963 -3.65541963 -3.65541963 -3.65541963
 -3.65541963 -3.65541963 -3.65541963 -3.65541963 -3.65541963 -3.65541963
 -3.65541963 -3.65541963 -3.65541963 -3.65541963 -3.65541963 -3.65541963
 -3.65541963 -3.65541963 -3.65541963]


In [24]:
sortedX = sorted_X /weights

X_to_denorm = sortedX[:,:-1]
# print(cluster_centers_to_denorm)
# Denormalize the cluster centers
X_denormalized = scaler.inverse_transform(X_to_denorm)

X_final= np.concatenate((X_denormalized,sortedX[:,-1].reshape(-1, 1)),axis=1)
X_final

array([[2.00000000e+00, 1.00000000e+00, 1.59000000e+06, 8.84000000e+02,
        2.40000000e+01, 6.59000000e+02, 1.86000000e+02, 3.00000000e+02,
        1.00000000e+00],
       [2.00000000e+00, 1.00000000e+00, 6.49000000e+05, 7.55000000e+02,
        2.90000000e+01, 4.92000000e+02, 2.12000000e+02, 2.46000000e+02,
        1.00000000e+00],
       [2.00000000e+00, 1.00000000e+00, 7.99000000e+05, 1.13686838e-13,
        2.50000000e+01, 5.43000000e+02, 2.09000000e+02, 2.72000000e+02,
        1.00000000e+00],
       [2.00000000e+00, 1.00000000e+00, 1.29500000e+06, 1.82099502e+03,
        2.30000000e+01, 4.93000000e+02, 1.85000000e+02, 2.78000000e+02,
        1.00000000e+00],
       [2.00000000e+00, 1.00000000e+00, 1.05000000e+06, 1.13686838e-13,
        2.30000000e+01, 6.83000000e+02, 1.76000000e+02, 2.94000000e+02,
        1.00000000e+00],
       [2.00000000e+00, 1.00000000e+00, 6.99000000e+05, 6.00000000e+02,
        2.00000000e+01, 4.53000000e+02, 1.68000000e+02, 2.16000000e+02,
        1.0

In [25]:
df_X=pd.DataFrame(X_final,columns=list(cols))
df_sorted=np.floor(df_X)
df_sorted=df_sorted.set_index(row_indexes)
df_sorted

,bedrooms,bathrooms,price,livingArea,leisure_within_5km,shops_within_5km,schools_within_5km,transit_within_2km,homeType
zpid,,,,,,,,,
61968154,2.0,1.0,1590000.0,884.0,24.0,659.0,186.0,300.0,1.0
31546758,2.0,1.0,649000.0,755.0,29.0,492.0,212.0,246.0,1.0
72532295,2.0,1.0,799000.0,0.0,25.0,543.0,209.0,272.0,1.0
2139218516,2.0,1.0,1295000.0,1820.0,23.0,493.0,185.0,278.0,1.0
31545643,2.0,1.0,1050000.0,0.0,23.0,683.0,176.0,294.0,1.0
97537243,2.0,1.0,698999.0,600.0,20.0,453.0,168.0,216.0,1.0
342149156,2.0,1.0,1250000.0,1100.0,21.0,619.0,161.0,323.0,1.0
72535178,2.0,1.0,1049000.0,0.0,24.0,350.0,185.0,216.0,1.0
121999838,2.0,1.0,625000.0,750.0,31.0,350.0,205.0,305.0,1.0


In [26]:
top_10_recommendations = df_sorted.head(10)
top_10_recommendations

,bedrooms,bathrooms,price,livingArea,leisure_within_5km,shops_within_5km,schools_within_5km,transit_within_2km,homeType
zpid,,,,,,,,,
61968154,2.0,1.0,1590000.0,884.0,24.0,659.0,186.0,300.0,1.0
31546758,2.0,1.0,649000.0,755.0,29.0,492.0,212.0,246.0,1.0
72532295,2.0,1.0,799000.0,0.0,25.0,543.0,209.0,272.0,1.0
2139218516,2.0,1.0,1295000.0,1820.0,23.0,493.0,185.0,278.0,1.0
31545643,2.0,1.0,1050000.0,0.0,23.0,683.0,176.0,294.0,1.0
97537243,2.0,1.0,698999.0,600.0,20.0,453.0,168.0,216.0,1.0
342149156,2.0,1.0,1250000.0,1100.0,21.0,619.0,161.0,323.0,1.0
72535178,2.0,1.0,1049000.0,0.0,24.0,350.0,185.0,216.0,1.0
121999838,2.0,1.0,625000.0,750.0,31.0,350.0,205.0,305.0,1.0


In [27]:
bleu_scorer = BLEU()

column = 'summary' ## Change the column name as needed,, make it 'summary' for the best model

bleu_scores = [bleu_scorer.sentence_score(
    hypothesis=description,
    references=[reference],
).score for reference in top_10_recommendations[column].tolist()]

bleu_scores

KeyError: 'summary'

In [ ]:
print(statistics.mean(bleu_scores))
print(statistics.variance(bleu_scores))

In [ ]:
rouge_scorer = Rouge()

rouge_scores = [rouge_scorer.get_scores(
    hyps=description,
    refs=reference,
)[0]["rouge-l"]["f"] for reference in top_10_recommendations[column].tolist()]
# score[0]["rouge-l"]["f"]

rouge_scores

In [ ]:
print(statistics.mean(rouge_scores))
print(statistics.variance(rouge_scores))